<a href="https://colab.research.google.com/github/kangwonlee/pytorch-ibm-coursera/blob/main/week04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Hello PyTorch 👋🏻



references
* https://www.coursera.org/learn/deep-neural-networks-with-pytorch/
* https://github.com/damounayman/Deep-Neural-Networks-with-PyTorch/blob/main/Week1/1D_tensors.ipynb



## week 4



### 6.1 Softmax Prediction



### 6.2 Softmax Function



### 6.3 Softmax PyTorch



## Shallow Neural Networks



### 7.1 Neural Networks in One Dimension



### 7.2 Neural Networks More Hidden Neurons



### 7.3 Neural Networks with Multiple Dimensional Input



### 7.4 Multi-Class Neural Networks



### 7.5 Backpropagation



### 7.6 Activation Function

